In [1]:
import pandas as pd

# Enoncé

Vous travaillez au sein de **LetsCom**, une entreprise de communication qui a pour objectif de créer des campagnes de publicité pour des entreprises tierces, dans le but d'inciter les gens à acheter des produits chez ces dîtes entreprises. Ainsi, plusieurs campagnes marketing ont été diffusées à 29 758 clients potentiels (une seule campagne par client) pour le compte d'une entreprise marketing **greenSCart**.


Plusieurs semaines sont passées, il est à présent temps d'analyser les performances des campagnes en termes de clics et d'achats. Votre rôle dans cette analyse va être de préparer le terrain, en formattant les données pour les rendre exploitables.

Vous avez pour cela trois fichiers à votre disposition :
- *campaign.csv* : un fichier présentant, par client, la campagne reçue et le canal d'acquisition (external_site_id)
- *click.csv* : un fichier présentant l'ensemble des clients ayant cliqué au moins une fois sur le lien contenu dans la campagne, avec la date du premier clic
- *purchase.csv* : un fichier présentant l'ensemble des clients ayant acheté parmi ceux ayant cliqué sur le lien, avec la date d'achat et une série d'informations sur le client.


# Exercice

## Partie 1 : analyse sur les campagnes

### 1. Importation 

Vous commencerez par importer le fichier campaign.csv. Pour cela, vous utiliserez la fonction [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).

In [2]:
campaign = pd.read_csv('campaign.csv')

La première étape après toute importation est de regarder les premières lignes, cela permet :
- de vérifier que l'importation s'est bien passée
- d'avoir un apperçu rapide de ce à quoi ressemble notre dataframe

In [3]:
campaign.head()

,client_id,campaign_id,external_site_id
0,2392769782,3,370668
1,2693949047,1,339897
2,6841408721,2,989339
3,1457891936,1,530191
4,5354259272,1,206900


La seconde chose à faire est une petite vérification sur les types de chaque colonne attribués par pandas lors de l'importation, encore une fois pour vérifier que l'importation s'est bien déroulée. Affichez le type de l'ensemble des colonnes

In [4]:
campaign.dtypes

client_id           int64
campaign_id         int64
external_site_id    int64
dtype: object

Combien de lignes et colonnes sont contenues dans notre fichier ?

In [5]:
campaign.shape

(29758, 3)

Modifiez le nom de la variable `external_site_id` par `canal_id`.

In [6]:
campaign = campaign.rename(columns={"external_site_id": "canal_id"})
campaign.head()

,client_id,campaign_id,canal_id
0,2392769782,3,370668
1,2693949047,1,339897
2,6841408721,2,989339
3,1457891936,1,530191
4,5354259272,1,206900


### 2. Méthodes de series

Il va être temps de commencer notre analyse en déterminant le nombre de campagnes différentes qui ont été envoyées. En premier lieu, sélectionnez uniquement la colonne `campaign_id`.

In [7]:
campaign['campaign_id']

0        3
1        1
2        2
3        1
4        1
        ..
29753    3
29754    1
29755    1
29756    1
29757    1
Name: campaign_id, Length: 29758, dtype: int64

Même si cette dernière est codée avec des nombres, il s'agit bien d'une variable **qualitative**. Nous verrons un peu plus tard comment changer le type, mais nous chercherons avant tout le nombre de valeurs différentes au sein de notre variable : trouvez une [méthode](https://openclassrooms.com/fr/courses/6204541-initiez-vous-a-python-pour-lanalyse-de-donnees/6237921-programmez-en-oriente-objet) appliquée à cette colonne permettant d'obtenir cela.

In [8]:
campaign['campaign_id'].unique()

array([3, 1, 2], dtype=int64)

De façon assez similaire, combien de canaux d'acquisition (`canal_id`) différents ont été utilisés ?

In [9]:
len(campaign['canal_id'].unique()) # ou campaign['canal_id'].unique().shape

12880

Nous souhaitons à présent avoir le nombre de personnes qui ont été contactées par campagne, en gardant bien en tête qu'un individu n'a pu recevoir qu'une seule campagne sur les trois envoyées, donc une ligne de notre dataframe correspond à un client unique

In [10]:
campaign['campaign_id'].value_counts()

1    10038
3     9863
2     9857
Name: campaign_id, dtype: int64

### 3. Ajout des informations de click et de purchase

Dans un premier temps, importez les fichiers *click.csv* et *purchase.csv*, et affichez les 5 premières lignes

In [11]:
click = pd.read_csv('click.csv')
click.head()

,click_date,client_id
0,2025-11-01 01:57:47,4217174943
1,2025-11-01 02:11:21,8581188253
2,2025-11-01 02:13:16,2693232025
3,2025-11-01 03:28:12,8905731989
4,2025-11-01 03:51:23,2071156755


In [12]:
purchase = pd.read_csv('purchase.csv')
purchase.head()

,purchase_date,client_id,product_id,gender,price,age
0,1763303606,7048447933,1,m,354,65
1,1763341126,9138057668,8,f,165,37
2,1763398855,1327443078,3,f,29,28
3,1763455517,1083555113,3,f,29,23
4,1763543415,665955021,6,f,34,25


Il va falloir à présent ajouter les informations de click et purchase à notre dataframe initial campaign. Nous pourrons faire cela via la fonction [`merge`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html) de pandas.

Il existe 3 grands type de jointures :
- interne
- externe
- à droite ou à gauche

Ici, on souhaite absolument conserver l'ensemble des informations qui sont dans notre dataframe initial (campaign). Faites dans un premier temps la jointure adaptée entre *campaign* et *click* :

In [13]:
df_final = pd.merge(campaign, click, on='client_id', how='left')
df_final.head()

,client_id,campaign_id,canal_id,click_date
0,2392769782,3,370668,NaN
1,2693949047,1,339897,NaN
2,6841408721,2,989339,NaN
3,1457891936,1,530191,NaN
4,5354259272,1,206900,NaN


Vous avez normalement des NaN qui apparaissent : pas de panique tout est parfaitement normal, car sur l'ensemble des clients, tous n'ont pas cliqué !

Faites ensuite la jointure entre le dataframe obtenu et la table purchase, de la même façon. Vous nommerez le dataframe final, `df_final` :

In [14]:
df_final = pd.merge(df_final, purchase, on='client_id', how='left')
df_final.head()

,client_id,campaign_id,canal_id,click_date,purchase_date,product_id,gender,price,age
0,2392769782,3,370668,NaN,NaN,NaN,NaN,NaN,NaN
1,2693949047,1,339897,NaN,NaN,NaN,NaN,NaN,NaN
2,6841408721,2,989339,NaN,NaN,NaN,NaN,NaN,NaN
3,1457891936,1,530191,NaN,NaN,NaN,NaN,NaN,NaN
4,5354259272,1,206900,NaN,NaN,NaN,NaN,NaN,NaN


### 4. Analyse rapide des informations de notre table

Utilisez les fonctions [`describe`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) et [`info`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html) pour avoir quelques informations rapides sur les variables de notre dataframe :

In [15]:
df_final.describe(include='all')

,client_id,campaign_id,canal_id,click_date,purchase_date,product_id,gender,price,age
count,2.975800e+04,29758.000000,29758.000000,1775,1.400000e+02,140.000000,140,140.000000,140.000000
unique,NaN,NaN,NaN,1772,NaN,NaN,2,NaN,NaN
top,NaN,NaN,NaN,2025-11-28 23:27:48,NaN,NaN,f,NaN,NaN
freq,NaN,NaN,NaN,2,NaN,NaN,87,NaN,NaN
mean,5.026531e+09,1.994119,501509.408058,NaN,1.765143e+09,4.328571,NaN,145.807143,39.085714
std,2.905848e+09,0.817771,289922.270987,NaN,8.263867e+05,2.190374,NaN,112.746478,15.167879
min,3.544420e+05,1.000000,45.000000,NaN,1.763304e+09,1.000000,NaN,29.000000,19.000000
25%,2.520960e+09,1.000000,249755.000000,NaN,1.764510e+09,3.000000,NaN,67.000000,27.000000
50%,5.042014e+09,2.000000,502512.000000,NaN,1.765181e+09,4.000000,NaN,89.000000,34.500000
75%,7.548261e+09,3.000000,753595.750000,NaN,1.765813e+09,6.000000,NaN,299.000000,56.000000


In [16]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29758 entries, 0 to 29757
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   client_id      29758 non-null  int64  
 1   campaign_id    29758 non-null  int64  
 2   canal_id       29758 non-null  int64  
 3   click_date     1775 non-null   object 
 4   purchase_date  140 non-null    float64
 5   product_id     140 non-null    float64
 6   gender         140 non-null    object 
 7   price          140 non-null    float64
 8   age            140 non-null    float64
dtypes: float64(4), int64(3), object(2)
memory usage: 2.3+ MB


A partir de cette dernière méthode, pouvez-vous déterminer le nombre de personnes ayant cliqué et le nombre de personnes ayant acheté parmi ces dernières ?

### 5. Changement de type

En regardant le résultat de la méthode *info*, on se rend compte que plusieurs variables n'ont pas le bon type, notamment : client_id, campaign_id, canal_id et product_id qui ont été importées en int ou float, alors que ce sont des variables qualitatives.

[Modifiez](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) donc leur type pour les mettre en string ou category.

In [17]:
var_modif = ['client_id', 'campaign_id', 'canal_id', 'product_id']

for var in var_modif:
    df_final[var] = df_final[var].astype("category")

On remarque également que click_date et purchase_date n'ont pas le bon format : ce sont censés être des dates, alors qu'elles sont actuellement codées comme étant respectivement chaine de caractère (`object`) et décimaux (`float`). La fonction utile dans ce cas sera [`to_datetime`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)

Convertissez click_date au format datetime :

In [18]:
df_final['click_date'] = pd.to_datetime(df_final['click_date'], format='%Y-%m-%d %H:%M:%S')

*purchase_date* est un peu particulière, car le format est ce qu'on appelle un timestamp. Le timestamp désigne le nombre de secondes écoulées depuis le 1er janvier 1970 à minuit UTC précises jusqu'à la date exprimée. Même si le format diffère, la fonction de conversion reste la même, il suffira de modifier les options de cette dernière.

In [19]:
df_final['purchase_date'] = pd.to_datetime(df_final['purchase_date'], unit='s')

In [20]:
df_final.dtypes

client_id              category
campaign_id            category
canal_id               category
click_date       datetime64[ns]
purchase_date    datetime64[ns]
product_id             category
gender                   object
price                   float64
age                     float64
dtype: object

### 6. la sélection au sein d'un dataframe

Nous avons d'ores et déjà vu comment sélectionner une colonne. En utilisant une écriture similaire, sélectionnez deux colonnes de votre choix.

In [21]:
df_final[['client_id', 'campaign_id']]

,client_id,campaign_id
0,2392769782,3
1,2693949047,1
2,6841408721,2
3,1457891936,1
4,5354259272,1
...,...,...
29753,2701723551,3
29754,7271996385,1
29755,4035966889,1
29756,7499403392,1


Il est à présent temps d'utiliser la fonction de sélection qui vous sera le plus utile, la méthode [`loc`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html). A partir de cette dernière, sélectionnez et déterminez le nombre de clients dont on a l'information, qui ont plus de 65 ans :

In [22]:
df_final.loc[df_final['age'] > 65, :]

,client_id,campaign_id,canal_id,click_date,purchase_date,product_id,gender,price,age
552,8751879875,1,429717,2025-11-01 15:49:13,2025-11-30 20:23:16,1.0,f,354.0,68.0
3109,5031077513,1,201592,2025-11-04 07:53:45,2025-12-01 19:01:50,8.0,f,165.0,67.0
7840,3755043429,2,335839,2025-11-09 02:11:04,2025-11-24 16:04:41,1.0,f,354.0,68.0
11728,7928559147,1,196935,2025-11-12 23:30:03,2025-12-03 21:39:40,1.0,f,354.0,66.0
12758,3179544440,1,970704,2025-11-14 01:27:15,2025-12-03 00:09:14,1.0,f,354.0,69.0


Sélectionnez uniquement les identifiants de ces clients :

In [23]:
df_final.loc[df_final['age'] > 65, 'client_id']

552      8751879875
3109     5031077513
7840     3755043429
11728    7928559147
12758    3179544440
Name: client_id, dtype: category
Categories (29758, int64): [354442, 522647, 668537, 692084, ..., 9998833142, 9998909223, 9999085476, 9999881734]

Sélectionnez à présent les clients de moins de 25 ans ayant acheté le produit 8 :

In [24]:
df_final.loc[(df_final['age'] < 25) & (df_final['product_id'] == 8), :]

,client_id,campaign_id,canal_id,click_date,purchase_date,product_id,gender,price,age
548,7331215946,2,866075,2025-11-01 15:35:54,2025-12-14 14:58:13,8.0,m,165.0,21.0
18629,9353335552,3,604738,2025-11-20 00:54:25,2025-12-14 23:05:32,8.0,f,165.0,20.0


Sur ces clients, sélectionnez uniquement leur identifiant et l'identifant de la campagne qu'ils ont reçue

In [25]:
df_final.loc[(df_final['age'] < 25) & (df_final['product_id'] == 8), ['client_id','campaign_id']]

,client_id,campaign_id
548,7331215946,2
18629,9353335552,3


Pour finir, vous créerez un nouveau dataframe, qui contiendra uniquement les 140 lignes qui correspondent à un achat client

In [33]:
achats = df_final.loc[df_final['purchase_date'].notnull(),:].copy()

Au sein de ce nouveau dataframe, sélectionnez les identifiants des clients ayant reçu la campagne selon l'un des canaux d'acquisition suivant : 

550488, 154492, 721316, 464778, 850899

In [34]:
canaux = [550488, 154492, 721316, 464778, 850899]

achats.loc[achats['canal_id'].isin(canaux),:]

,client_id,campaign_id,canal_id,click_date,purchase_date,product_id,gender,price,age
9409,3670039263,1,550488,2025-11-10 17:06:23,2025-11-29 03:47:52,7.0,f,109.0,38.0
9770,675653663,1,154492,2025-11-10 23:43:52,2025-11-30 23:08:08,5.0,f,89.0,39.0
10025,8300306275,2,721316,2025-11-11 06:59:20,2025-11-29 03:07:18,4.0,f,299.0,62.0
10797,9432221904,2,464778,2025-11-12 00:17:23,2025-12-11 18:41:43,8.0,m,165.0,36.0
11092,3745071281,2,850899,2025-11-12 08:03:08,2025-11-29 22:42:27,4.0,m,299.0,54.0


## Partie 2 : analyse sur les achats

Pour la suite de l'exercice, on se concentrera uniquement sur le dernier dataframe créé contenant les informations d'achat

### 1. Création de nouvelles variables

Le prix actuellement indiqué correspond au prix hors taxe. Créez une nouvelle variable correspondant au prix TTC de chaque produit vendu, en sachant que :

$$Prix  TTC = 1.2 * Prix  HT$$

In [41]:
achats['price TTC'] = achats['price'] * 1.2

Il serait également intéressant d'extraire uniquement le jour de l'achat (au format `date`), à partir de la date d'achat pour éventuellement calculer le chiffre d'affaire total par jour.

In [42]:
achats['jour_achat'] = achats['purchase_date'].dt.date

In [43]:
achats.head()

,client_id,campaign_id,canal_id,click_date,purchase_date,product_id,gender,price,age,price HT,jour_achat,price TTC
11,665955022,2,646537,2025-11-02 01:06:55,2025-11-21 13:10:51,1.0,f,354.0,63.0,424.8,2025-11-21,424.8
51,2693232025,2,673707,2025-11-01 02:13:16,2025-11-27 07:27:54,4.0,f,299.0,56.0,358.8,2025-11-27,358.8
87,7847011933,2,276236,2025-11-27 04:47:31,2025-12-25 05:01:34,3.0,f,29.0,19.0,34.8,2025-12-25,34.8
174,7048447933,3,182313,2025-11-01 06:39:02,2025-11-16 14:33:26,1.0,m,354.0,65.0,424.8,2025-11-16,424.8
302,9554597531,3,341237,2025-11-01 10:00:16,2025-12-01 01:15:28,5.0,f,89.0,37.0,106.8,2025-12-01,106.8


### 2. agrégation

A partir de cette variable nouvellement créée et de la fonction [`groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html), créez un nouveau dataframe comportant le chiffre d'affaire TTC total quotidien.

In [47]:
achats.groupby('jour_achat').sum()[['price TTC']]

,price TTC
jour_achat,
2025-11-16,424.8
2025-11-17,232.8
2025-11-18,34.8
2025-11-19,40.8
2025-11-20,106.8
2025-11-21,424.8
2025-11-22,106.8
2025-11-23,393.6
2025-11-24,729.6
